In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
spark

In [63]:
pwd

'/home/hadoop'

In [3]:
hflight = spark.read.option('inferSchema','true').\
    option('header','true').\
    csv("file:///home/hadoop/hflight.csv")

In [65]:
hflight.limit(2).show()

+----+-----+----------+---------+-------+-------+-------------+---------+-------+-----------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|ArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|
+----+-----+----------+---------+-------+-------+-------------+---------+-------+-----------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+
|2011|    1|         1|        6|   1400|   1500|           AA|      428| N576AA|               60|     40|     -10|       0|   IAH| DFW|     224|     7|     13|        0|            null|       0|
|2011|    1|         2|        7|   1401|   1501|           AA|      428| N557AA|               60|     45|      -9|       1|   IAH| DFW|     224|     6|      9|        0|            null|       0|
+----+----

In [66]:
hflight.count()

227496

In [67]:
from pyspark.sql.functions import avg, stddev,col

In [68]:
hflight.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)



In [69]:
hflight  = hflight.withColumn("ArrDelay",col("ArrDelay").cast("Integer"))
hflight = hflight.withColumn("DepDelay",col("DepDelay").cast("Integer"))
# agg에 자동적으로 변환해주는게 있지않을까요?

In [70]:
hflight.groupby("Origin","Dest").avg("ArrDelay","DepDelay").show()
# 순수한 RDD 보다 DF를 사용하는게 더 빠르다 하지만 내부적으론 다 RDD를 사용된다

+------+----+------------------+------------------+
|Origin|Dest|     avg(ArrDelay)|     avg(DepDelay)|
+------+----+------------------+------------------+
|   HOU| PHL|23.541666666666668| 33.19124423963134|
|   IAH| LIT| 6.764063811922754| 7.740771812080537|
|   IAH| GSP| 9.617359413202934|  9.08272506082725|
|   IAH| DEN| 7.441828254847645| 10.37734288864388|
|   IAH| TUS| 7.801679586563307| 7.783870967741936|
|   IAH| ABQ| 7.928692699490663| 6.703891708967851|
|   HOU| LIT| 3.533333333333333|7.4404432132963985|
|   IAH| ANC|26.080645161290324|            24.952|
|   IAH| SMF|  4.66271018793274| 9.093966369930762|
|   HOU| TUL| 7.855238095238096|11.208174904942966|
|   HOU| MDW|3.8126815101645692|13.204150579150578|
|   HOU| OKC| 9.441897233201582|12.509865824782953|
|   IAH| DSM|15.951104100946372|12.796850393700787|
|   IAH| DFW| 5.510741008930727| 8.764620938628159|
|   IAH| BOS| 6.559045956951716|10.519141531322505|
|   HOU| MAF| 6.918367346938775|10.507042253521126|
|   IAH| RIC

In [71]:
# hflight.groupby("Origin", "Dest").agg( avg("ArrDelay").alias("출발 지연 평균"),\
#                                        avg("DepDelay").alias("도착 지연 평균")).\
#     show()

In [72]:
# hflight.groupby("Origin","Dest").agg(avg("ArrDelay").alias("출발 지연 평균")).\
#     agg(avg("DepDelay").alias("도착 지연 평균")).\
#     show()

In [73]:
hflight.selectExpr("ArrDelay as `지연`").show()

+----+
|지연|
+----+
| -10|
|  -9|
|  -8|
|   3|
|  -3|
|  -7|
|  -1|
| -16|
|  44|
|  43|
|  29|
|   5|
|  -9|
|  -6|
| -11|
|  -1|
|  84|
|  -2|
|  -7|
|  72|
+----+
only showing top 20 rows



In [93]:
# hflight[hflight["ArrDelay"]>=5].groupby("Dest").count().sort('count', ascending=False).show()
# hflight[hflight["ArrDelay"]>=5].groupby("Dest").count().filter('count'>2000).show()

In [5]:
from pyspark.sql.functions import *

In [6]:
hflight.filter(col("ArrDelay")>=5).groupby("Dest").count().filter(col('count')>=2000).show()

+----+-----+
|Dest|count|
+----+-----+
| MSY| 2361|
| DEN| 2315|
| DAL| 3360|
| ATL| 2657|
| ORD| 2220|
| LAX| 2641|
+----+-----+



In [7]:
from pyspark.sql import SparkSession

# spark = SparkSession.builder.getOrCreate()
df = spark.read \
    .format("jdbc") \
    .option("driver","com.mysql.cj.jdbc.Driver") \
    .option("url", "jdbc:mysql://ip:port/encore_web") \
    .option("dbtable","encore_question") \
    .option("numPartitions",5) \
    .option("user", "id") \
    .option("password", 'password') \
    .load()




# 문제 2
-  목적지 공항에 대해 연착 건수를 구하고, 
-  연착 건수가 2000회 이상인 공항에 대한 데이터만 추출 
-  col -> Dest :목적지 공항 ArrDelay (연착은 5분이상)


hdfs -> 저장까지!!! 
csv, parquet

In [18]:
from pyspark.sql.functions import expr
# hflight.withColumn("ArrDelay", expr("ArrDelay == 0")).show()
# hflight.where(expr("ArrDelay == 0")).show()

In [8]:
from pyarrow import fs
import pyarrow as pa
import subprocess

classpath = subprocess.Popen(["/home/hadoop/hadoop/bin/hdfs", "classpath", "--glob"], stdout=subprocess.PIPE).communicate()[0]
os.environ["CLASSPATH"] = classpath.decode("utf-8")
hdfs = fs.HadoopFileSystem(host='ip', port=port, user='hadoop')


2024-06-19 02:06:41,980 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
df3 = hflight.filter(col("ArrDelay")>=5).groupby("Dest").count().filter(col('count')>=2000)

In [15]:
df3

DataFrame[Dest: string, count: bigint]

In [19]:
df3.show()

+----+-----+
|Dest|count|
+----+-----+
| MSY| 2361|
| DEN| 2315|
| DAL| 3360|
| ATL| 2657|
| ORD| 2220|
| LAX| 2641|
+----+-----+



In [16]:
df3.write.csv("hdfs://namenode:port/20240619/mort/my3.csv")

In [20]:
target = hflight.withColumn("count", expr("ArrDelay >=5")).\
    where(expr("count == True")).\
    groupby("Dest").\
    count().\
    filter(expr("count >= 2000")).select("Dest").rdd.flatMap(lambda x : x).collect()


hflight2 = hflight.where(col('Dest').isin(target))


In [23]:
# hflight2.show()
hflight2.write.csv("hdfs://namenode:port/20240619/mort/hflight2.csv")

In [27]:
# hflight2.show()
hflight2.write.parquet("hdfs://namenode:port/20240619/mort/hflight3.csv")

In [150]:

hflight2.select("Dest").distinct().show()


+----+
|Dest|
+----+
| MSY|
| DEN|
| DAL|
| ATL|
| ORD|
| LAX|
+----+



In [28]:
emp = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]
empColumns = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]

empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()
empDF.show(truncate=False)

dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.printSchema()
deptDF.show(truncate=False)


root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1    |
+------+--------+---------------+-----------+-----------+------+-----

In [30]:
empDF.join(deptDF,empDF.emp_dept_id == deptDF.dept_id , "inner").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



## 문제 3 
-  위의 결과를 바탕으로 목적지 공항 별 결항 횟수, 회항 횟수
-  운항 횟수를 구하시오 (Cancelled, Diverted, Air)
-  운항 횟수는 결항과 회항을 제외한 것 


In [31]:
hflight.limit(1).show()

+----+-----+----------+---------+-------+-------+-------------+---------+-------+-----------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|ArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|
+----+-----+----------+---------+-------+-------+-------------+---------+-------+-----------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+
|2011|    1|         1|        6|   1400|   1500|           AA|      428| N576AA|               60|     40|     -10|       0|   IAH| DFW|     224|     7|     13|        0|            null|       0|
+----+-----+----------+---------+-------+-------+-------------+---------+-------+-----------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+



In [148]:
hflight.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)



In [154]:
hflight4 = spark.read.parquet("hdfs://namenode:port/20240619/scw/hflight_ex3.parquet")

In [155]:
# hflight.groupby("Dest").count().limit(1).show()
a = hflight4.groupby("Dest").count()

In [156]:
# hflight.groupby("Dest").sum().select("Dest","sum(Cancelled)" ,"sum(Diverted)").limit(1).show()
b = hflight4.groupby("Dest").sum().select("Dest","sum(Cancelled)" ,"sum(Diverted)").withColumn("Dest1",col("Dest")).drop("Dest")
# b = b.withColumn("Dest1",col("Dest"))
# b = b.drop("Dest")

In [158]:
# hflight4.printSchema()

In [132]:
# hflight.groupby("Dest").sum().select("Dest","sum(Cancelled)" ,"sum(Diverted)").limit(1).show()
# b = hflight.groupby("Dest").sum().select("Dest","sum(Cancelled)" ,"sum(Diverted)").withColumn("Dest1",col("Dest")).drop("Dest")
# b = b.withColumn("Dest1",col("Dest"))
# b = b.drop("Dest")

In [159]:
c = a.join( b, a.Dest == b.Dest1, "inner").withColumn("answer",col("count") - col("sum(Diverted)") - col("sum(Cancelled)")).select("Dest","answer").show()
# c.show()

+----+------+
|Dest|answer|
+----+------+
| MSY|  6780|
| DEN|  5868|
| DAL|  9351|
| ATL|  7717|
| ORD|  5634|
| LAX|  6017|
+----+------+



In [73]:
# c.printSchema()

In [77]:
# c  = c.withColumn("count",col("count").cast("Integer"))
# c = c.withColumn("sum(Cancelled)",col("sum(Cancelled)").cast("Integer"))
# c = c.withColumn("sum(Diverted)",col("sum(Diverted)").cast("Integer"))

In [78]:
# c.printSchema()

In [143]:
# c = c.withColumn("answer",col("count").cast("Integer") - col("sum(Diverted)").cast("Integer") - col("sum(Cancelled)").cast("Integer")).select("Dest","answer").show()

In [144]:
# c.select("Dest","answer").show()

In [166]:
from pyspark.sql.functions import expr
url = "jdbc:mysql://ip:port/encore_web"
properties = {
    "user": "id",
    "password": 'password',
    "driver": "com.mysql.cj.jdbc.Driver"
}

a = hflight4.groupby("Dest").sum("Cancelled")
b = hflight4.groupby("Dest").sum("Diverted")
c = hflight4.groupby("Dest").count() 



In [168]:
a.join(b, on="Dest", how='left').\
    join(c, on="Dest", how='left').\
    withColumnRenamed("sum(Cancelled)", "Cancelled").\
    withColumnRenamed("sum(Diverted)", "Diverted").\
    withColumn("air", expr("count - Cancelled - Diverted")).\
    drop("count").\
    write.jdbc(url=url, table='mort', mode='append', properties=properties)
